### 생산자-소비자 문제
대표적인 프로세스 동기화에 대한 문제<br><br>
**생산자가 데이터를 생성**하면 **소비자는 그것을 소비**하는 형태에서 발생하는 문제를 말한다. <br>
컴퓨터 세계에서 예를 들면 웹 서버와 웹 클라이언트로 들 수 있다.<br>웹 서버가 데이터를 생산하여 웹에 관련되어 보여주는 작업들을 수행하고 웹 클라이언트는 웹 주소로 접속해 화면을 통해 보게 되는 형태의 소비 작용을 한다.



[참고](https://copycode.tistory.com/67)

## [수신된 이메일을 장기 보관하기 위해 처리하는 프로그램]
### '리스트'를 생산자-소비자 큐로 사용
- 생산자 함수: append 메소드 사용
- 소비자 함수: pop메소드 사용

In [12]:
import random

# walus 연산자 때문에 파이썬 3.8이상에서만 실행됨

class Email:
    def __init__(self, sender, receiver, message):
        self.sender = sender
        self.receiver = receiver
        self.message = message

    def __str__(self):
        return f"Email({self.sender},{self.receiver},{self.message})"

class NoEmailError(Exception):
    pass

# 이메일을 수신하는 역할을 한다.
def try_receive_email():
    # Email 인스턴스를 하나 반환한거나, NoEmailError를 발생시킨다
    # 99%확률로 Email을 발생시킴
    if (i := random.randint(0,100)) > 99:   # 100이면 True, 0~99이면 False
        raise NoEmailError
    else:
        return Email(f'홍길동{i}:','성춘향','아니 자네가 왜 여기에!')

# 생산자 함수
# 이메일을 받아서 나중에 소비될 수 있도록 큐에 넣는다.
def produce_emails(queue):
    while True:
        try:
            email = try_receive_email()
        except NoEmailError:
            return
        else:
            queue.append(email)  # 생산자

# 소비자 함수
# 이메일을 읽는다.(소비한다)
def consume_one_email(queue):
    if not queue:
        return
    email = queue.pop(0) # 소비자
    print(f"read email: {email}")

# 생산자-소비자 함수를 엮어준다.
# keep_running함수가 False를 반환할 때까지 생산-소비를 번갈아 반복한다.
def loop(queue, keep_running):
    while keep_running():
        produce_emails(queue)
        consume_one_email(queue)

In [13]:
# 99%확률로 True를 반환
def my_end_func():
    return random.randint(0, 100) > 1     # 0,1이면 False, 2~100이면 True
                                           # 의문: randint(1,100)이어야 99%확률인거 아닌가?

# my_end_func가 False를 반환 할 때까지 반복
loop([], my_end_func)

read email: Email(홍길동65:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동64:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동23:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동89:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동19:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동5:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동99:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동10:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동49:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동51:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동84:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동45:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동73:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동25:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동59:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동96:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동51:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동95:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동95:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동33:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동71:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동68:,성춘향,아니 자네가 왜 여기에!)
read email: Email(홍길동36:,성춘향,아니 자

하지만 크기가 늘어나면 리스트타입의 성능은 선형보다 더 나빠지기 때문에 문제가 발생할 수 있다..<br>
**리스트를 큐로 사용할 때 성능**이 어떤지 분석하기 위해 '마이크로 벤치마크'를 수행한다.
### '마이크로 벤치마크'란?
- 성능 테스트(Benchmarking)의 방식 중 하나입니다
- **매우 작은 단위의 성능을 테스트**하기 위해 고안되었습니다
- 작은 단위의 업무나 특정 알고리즘의 간접적인 처리시간(overhead)를 구합니다

> **'벤치마크'**는 컴퓨팅에서 특정 오브젝트에 대해 일반적으로 수많은 표준 테스트와 시도를 수행함으로써 오브젝트의 상대적인 성능 측정을 목적으로 컴퓨터 프로그램을 실행하는 행위

<br>



여러크기의 리스트를 사용해 이 벤치마크 함수를 실행하면 **데이터 크기와 성능의 관계**를 비교할 수 있다.

### 1) 리스트의 <u>append메소드</u>를 사용해 새로운 원소를 큐에 추가하는 연산의 성능을 벤치마크(테스트)
결과 분석)
- 데이터크기가 커짐에 따라 큐에 데이터를 넣는데 걸리는 전체 시간이 선형적으로 늘어난다. O(n)

In [3]:
import timeit
# timeit모듈을 사용해 마이크로 벤치마크를 수행

def print_results(count, tests):
    avg_iteration = sum(tests) / len(tests)
    print(f'\n원소 수: {count:>5,} 걸린시간: {avg_iteration:.6f}초')
    return count, avg_iteration

# 리스트의 append메소드 사용
def list_append_benchmark(count):
    def run(queue):
        for i in range(count):
            queue.append(i)

    tests = timeit.repeat(
        setup='queue = []',
        stmt='run(queue)',
        globals=locals(),
        repeat=1000,
        number=1)

    return print_results(count, tests)

def print_delta(before, after):
    before_count, before_time = before
    after_count, after_time = after
    growth = 1 + (after_count - before_count) / before_count
    slowdown = 1 + (after_time - before_time) / before_time
    print(f'데이터 크기 {growth:>4.1f}배, 걸린 시간 {slowdown:>4.1f}배')

baseline = list_append_benchmark(500)
for count in (1_000, 2_000, 3_000, 4_000, 5_000):
    comparison = list_append_benchmark(count)
    print_delta(baseline, comparison)


원소 수:   500 걸린시간: 0.000116초

원소 수: 1,000 걸린시간: 0.000131초
데이터 크기  2.0배, 걸린 시간  1.1배

원소 수: 2,000 걸린시간: 0.000250초
데이터 크기  4.0배, 걸린 시간  2.1배

원소 수: 3,000 걸린시간: 0.000322초
데이터 크기  6.0배, 걸린 시간  2.8배

원소 수: 4,000 걸린시간: 0.000594초
데이터 크기  8.0배, 걸린 시간  5.1배

원소 수: 5,000 걸린시간: 0.000590초
데이터 크기 10.0배, 걸린 시간  5.1배


### 2) 큐의 맨 앞에서 원소를 제거하는 <u>pop(0)</u> 호출

결과 분석)

- 큐 길이가 늘어남에 따라 **큐 길이의 제곱에 비례**해 늘어난다. O(n^2)
- pop(0)을 하면 **리스트의 모든 남은 원소를 제위치로 옮겨야 하므로,** 결과적으로 전체 리스트내용을 재대입하기 때문이다.

In [4]:
def list_pop_benchmark(count):
    def prepare():
        return list(range(count))

    def run(queue):
        while queue:
            queue.pop(0)

    tests = timeit.repeat(
        setup='queue = prepare()',
        stmt='run(queue)',
        globals=locals(),
        repeat=1000,
        number=1)

    return print_results(count, tests)

baseline = list_pop_benchmark(500)

for count in (1_000, 2_000, 3_000, 4_000, 5_000):
    comparison = list_pop_benchmark(count)
    print_delta(baseline, comparison)


원소 수:   500 걸린시간: 0.000134초

원소 수: 1,000 걸린시간: 0.000237초
데이터 크기  2.0배, 걸린 시간  1.8배

원소 수: 2,000 걸린시간: 0.000781초
데이터 크기  4.0배, 걸린 시간  5.8배

원소 수: 3,000 걸린시간: 0.001244초
데이터 크기  6.0배, 걸린 시간  9.3배

원소 수: 4,000 걸린시간: 0.002169초
데이터 크기  8.0배, 걸린 시간 16.2배

원소 수: 5,000 걸린시간: 0.003377초
데이터 크기 10.0배, 걸린 시간 25.2배


### 파이썬 collections 내장모듈의 deque클래스를 활용하자.

deque는 double-ended queue(양방향 큐)

In [5]:
import collections

def deque_append_benchmark(count):
    def prepare():
        return collections.deque()

    def run(queue):
        for i in range(count):
            queue.append(i)

    tests = timeit.repeat(
        setup='queue = prepare()',
        stmt='run(queue)',
        globals=locals(),
        repeat=1000,
        number=1)
    return print_results(count, tests)

baseline = deque_append_benchmark(500)
for count in (1_000, 2_000, 3_000, 4_000, 5_000):
    comparison = deque_append_benchmark(count)
    print_delta(baseline, comparison)


원소 수:   500 걸린시간: 0.000132초

원소 수: 1,000 걸린시간: 0.000112초
데이터 크기  2.0배, 걸린 시간  0.8배

원소 수: 2,000 걸린시간: 0.000244초
데이터 크기  4.0배, 걸린 시간  1.8배

원소 수: 3,000 걸린시간: 0.000392초
데이터 크기  6.0배, 걸린 시간  3.0배

원소 수: 4,000 걸린시간: 0.000563초
데이터 크기  8.0배, 걸린 시간  4.3배

원소 수: 5,000 걸린시간: 0.000728초
데이터 크기 10.0배, 걸린 시간  5.5배


deque의 popleft를 사용하면 대기열 길이에 선형적으로 비례해 시간이 늘어난다. O(n)

In [6]:
def dequeue_popleft_benchmark(count):
    def prepare():
        return collections.deque(range(count))

    def run(queue):
        while queue:
            queue.popleft()

    tests = timeit.repeat(
        setup='queue = prepare()',
        stmt='run(queue)',
        globals=locals(),
        repeat=1000,
        number=1)

    return print_results(count, tests)

baseline = dequeue_popleft_benchmark(500)
for count in (1_000, 2_000, 3_000, 4_000, 5_000):
    comparison = dequeue_popleft_benchmark(count)
    print_delta(baseline, comparison)


원소 수:   500 걸린시간: 0.000070초

원소 수: 1,000 걸린시간: 0.000109초
데이터 크기  2.0배, 걸린 시간  1.6배

원소 수: 2,000 걸린시간: 0.000221초
데이터 크기  4.0배, 걸린 시간  3.2배

원소 수: 3,000 걸린시간: 0.000328초
데이터 크기  6.0배, 걸린 시간  4.7배

원소 수: 4,000 걸린시간: 0.000512초
데이터 크기  8.0배, 걸린 시간  7.3배

원소 수: 5,000 걸린시간: 0.000626초
데이터 크기 10.0배, 걸린 시간  9.0배
